In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from Utils import test_env, utils
from Utils import HP_tuning 
from Utils.supervised import *

from RelationalModule import AC_networks as nets
from RelationalModule.MLP_AC_networks import Actor

from importlib import reload

parameters to try out and their priors, class for evaluation, model

In [3]:
reload(HP_tuning)

<module 'Utils.HP_tuning' from '/m/home/home9/94/dainesn1/unix/Workdir/RelationalDeepRL/Utils/HP_tuning.py'>

In [4]:
class EvalSandboxModel():
    def __init__(self, model, model_spec, game_params, n_epochs=50, n_samples=10000):
        self.model = model
        self.model_spec = model_spec
        self.game_params = game_params
        self.n_epochs = n_epochs
        self.n_samples = n_samples
        
    def evaluate_params(self, HPs, lambdas):
        train_V_lambda = [] 
        val_V_lambda = []
        for lr in lambdas:
            net = self.model(**self.model_spec, **HPs)
            results = supervised_training(net, lr, self.n_epochs, self.n_samples, 
                                          self.game_params, get_probs=True)
            trained_net, train_loss, val_loss, dataloader_dict, state_set, action_set, env = results
            
            train_V = 1-np.array(train_loss)
            train_V_lambda.append(train_V)
            
            val_V = 1-np.array(val_loss)
            val_V_lambda.append(val_V)
            
        return np.array(train_V_lambda), np.array(val_V_lambda)

In [5]:
# Variable parameters
X = 10
Y = 10
initial = [0,0]
goal = [2,2]
MAX_STEPS = 25

game_params = dict(x=X, y=Y, initial=initial, goal=goal, max_steps=MAX_STEPS, 
                   greyscale_state=True, return_ohe=True)

In [6]:
model = nets.MultiplicativeActor
model_spec = dict(action_space=4, linear_size = X+2, in_channels=3)

In [11]:
eval_instance = EvalSandboxModel(model, model_spec, game_params, 50, 10000)

In [15]:
value_priors_dict = {'out_channels':([[6],[12],[6,12]], [1/3, 1/3, 1/3]),
                     'max_pool_size':([2,3,4], [0.5, 0.3, 0.2]),
                     'n_features':([16,32,64],[1/3,1/3,1/3]),
                     'info_channels':([4,6,8],[0.2, 0.3, 0.5]),
                     'mask_channels':([4,6,8],[0.2, 0.3, 0.5]),
                     'hidden_channels':([6, 12, 32], [1/3, 1/3, 1/3]),
                     'residual_hidden_dim':([16,32,64],[1/3, 1/3, 1/3]),
                     'n_residual_layers':([1,2,3,4],[0.1, 0.3, 0.3, 0.3])}

In [16]:
bayes_HP_tuning = HP_tuning.BayesHPTuning(value_priors_dict, eval_instance)

In [17]:
for i in range(10):
    bayes_HP_tuning.step()


Configuration sampled: 
	out_channels :  [12]
	max_pool_size :  4
	n_features :  64
	info_channels :  6
	mask_channels :  4
	hidden_channels :  6
	residual_hidden_dim :  64
	n_residual_layers :  4

Creating dataset...

Training network...
Verbose:  True
Using cuda:  False
Epoch 1, 10% 	 Train loss: 0.2955 took: 0.18s  

/m/work/modules/automatic/anaconda/envs/aalto-ubuntu1804-generic/software/anaconda/2020-01-tf2/5a34a04a/lib/python3.7/site-packages/torch/nn/functional.py:1946: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


Epoch 1, 100% 	 Train loss: 0.2614 took: 1.85s  Val. loss: 0.2699
Epoch 2, 100% 	 Train loss: 0.2420 took: 1.85s  Val. loss: 0.2089
Epoch 3, 100% 	 Train loss: 0.1941 took: 1.86s  Val. loss: 0.2072
Epoch 4, 100% 	 Train loss: 0.1884 took: 1.85s  Val. loss: 0.2021
Epoch 5, 100% 	 Train loss: 0.1866 took: 1.85s  Val. loss: 0.2064
Epoch 6, 100% 	 Train loss: 0.1838 took: 1.85s  Val. loss: 0.2032
Epoch 7, 100% 	 Train loss: 0.1838 took: 1.86s  Val. loss: 0.1980
Epoch 8, 100% 	 Train loss: 0.1829 took: 1.84s  Val. loss: 0.2020
Epoch 9, 100% 	 Train loss: 0.1818 took: 1.84s  Val. loss: 0.2013
Epoch 10, 100% 	 Train loss: 0.1807 took: 1.86s  Val. loss: 0.1979
Epoch 11, 100% 	 Train loss: 0.1797 took: 1.84s  Val. loss: 0.1980
Epoch 12, 100% 	 Train loss: 0.1800 took: 1.85s  Val. loss: 0.2017
Epoch 13, 100% 	 Train loss: 0.1794 took: 1.86s  Val. loss: 0.2003
Epoch 14, 100% 	 Train loss: 0.1785 took: 1.84s  Val. loss: 0.1975
Epoch 15, 100% 	 Train loss: 0.1773 took: 1.85s  Val. loss: 0.1931
Epoc

Epoch 20, 100% 	 Train loss: 0.1890 took: 1.86s  Val. loss: 0.1903
Epoch 21, 100% 	 Train loss: 0.1876 took: 1.85s  Val. loss: 0.1993
Epoch 22, 100% 	 Train loss: 0.1871 took: 1.86s  Val. loss: 0.1897
Epoch 23, 100% 	 Train loss: 0.1826 took: 1.86s  Val. loss: 0.1826
Epoch 24, 100% 	 Train loss: 0.1821 took: 1.85s  Val. loss: 0.1834
Epoch 25, 100% 	 Train loss: 0.1792 took: 1.85s  Val. loss: 0.1845
Epoch 26, 100% 	 Train loss: 0.1778 took: 1.86s  Val. loss: 0.1838
Epoch 27, 100% 	 Train loss: 0.1780 took: 1.88s  Val. loss: 0.1781
Epoch 28, 100% 	 Train loss: 0.1731 took: 1.86s  Val. loss: 0.1785
Epoch 29, 100% 	 Train loss: 0.1697 took: 1.86s  Val. loss: 0.1777
Epoch 30, 100% 	 Train loss: 0.1711 took: 1.89s  Val. loss: 0.1754
Epoch 31, 100% 	 Train loss: 0.1663 took: 1.88s  Val. loss: 0.1711
Epoch 32, 100% 	 Train loss: 0.1630 took: 1.90s  Val. loss: 0.1695
Epoch 33, 100% 	 Train loss: 0.1629 took: 1.89s  Val. loss: 0.1673
Epoch 34, 100% 	 Train loss: 0.1594 took: 1.90s  Val. loss: 0.

Epoch 29, 100% 	 Train loss: 0.1929 took: 0.98s  Val. loss: 0.1861
Epoch 30, 100% 	 Train loss: 0.1900 took: 0.99s  Val. loss: 0.1846
Epoch 31, 100% 	 Train loss: 0.1896 took: 1.39s  Val. loss: 0.1812
Epoch 32, 100% 	 Train loss: 0.1884 took: 1.74s  Val. loss: 0.1814
Epoch 33, 100% 	 Train loss: 0.1886 took: 1.75s  Val. loss: 0.1846
Epoch 34, 100% 	 Train loss: 0.1858 took: 1.73s  Val. loss: 0.1789
Epoch 35, 100% 	 Train loss: 0.1847 took: 1.74s  Val. loss: 0.1774
Epoch 36, 100% 	 Train loss: 0.1838 took: 1.75s  Val. loss: 0.1801
Epoch 37, 100% 	 Train loss: 0.1821 took: 1.73s  Val. loss: 0.1806
Epoch 38, 100% 	 Train loss: 0.1806 took: 1.74s  Val. loss: 0.1769
Epoch 39, 100% 	 Train loss: 0.1813 took: 1.72s  Val. loss: 0.1728
Epoch 40, 100% 	 Train loss: 0.1795 took: 1.74s  Val. loss: 0.1745
Epoch 41, 100% 	 Train loss: 0.1766 took: 1.74s  Val. loss: 0.1688
Epoch 42, 100% 	 Train loss: 0.1746 took: 1.74s  Val. loss: 0.1685
Epoch 43, 100% 	 Train loss: 0.1725 took: 1.76s  Val. loss: 0.

...

Epoch 19, 100% 	 Train loss: 0.1929 took: 1.96s  Val. loss: 0.2011
Epoch 20, 100% 	 Train loss: 0.1930 took: 1.93s  Val. loss: 0.2023
Epoch 21, 100% 	 Train loss: 0.1937 took: 1.95s  Val. loss: 0.1987
Epoch 22, 100% 	 Train loss: 0.1936 took: 1.95s  Val. loss: 0.2026
Epoch 23, 100% 	 Train loss: 0.1926 took: 1.96s  Val. loss: 0.1991
Epoch 24, 100% 	 Train loss: 0.1936 took: 1.97s  Val. loss: 0.1995
Epoch 25, 100% 	 Train loss: 0.1916 took: 1.96s  Val. loss: 0.1970
Epoch 26, 100% 	 Train loss: 0.1906 took: 1.96s  Val. loss: 0.1966
Epoch 27, 100% 	 Train loss: 0.1912 took: 1.96s  Val. loss: 0.1964
Epoch 28, 100% 	 Train loss: 0.1905 took: 1.97s  Val. loss: 0.1987
Epoch 29, 100% 	 Train loss: 0.1902 took: 1.97s  Val. loss: 0.1991
Epoch 30, 100% 	 Train loss: 0.1904 took: 1.96s  Val. loss: 0.1955
Epoch 31, 100% 	 Train loss: 0.1883 took: 1.97s  Val. loss: 0.1948
Epoch 32, 100% 	 Train loss: 0.1874 took: 1.96s  Val. loss: 0.1944
Epoch 33, 100% 	 Train loss: 0.1880 took: 1.97s  Val. loss: 0.

Epoch 28, 100% 	 Train loss: 0.1218 took: 1.88s  Val. loss: 0.1332
Epoch 29, 100% 	 Train loss: 0.1188 took: 1.41s  Val. loss: 0.1276
Epoch 30, 100% 	 Train loss: 0.1178 took: 1.89s  Val. loss: 0.1215
Epoch 31, 100% 	 Train loss: 0.1118 took: 1.91s  Val. loss: 0.1201
Epoch 32, 100% 	 Train loss: 0.1109 took: 1.91s  Val. loss: 0.1198
Epoch 33, 100% 	 Train loss: 0.1107 took: 1.91s  Val. loss: 0.1142
Epoch 34, 100% 	 Train loss: 0.1038 took: 1.95s  Val. loss: 0.1145
Epoch 35, 100% 	 Train loss: 0.1029 took: 1.96s  Val. loss: 0.1101
Epoch 36, 100% 	 Train loss: 0.1015 took: 1.95s  Val. loss: 0.1114
Epoch 37, 100% 	 Train loss: 0.0994 took: 1.96s  Val. loss: 0.1085
Epoch 38, 100% 	 Train loss: 0.0976 took: 1.98s  Val. loss: 0.1048
Epoch 39, 100% 	 Train loss: 0.0947 took: 1.97s  Val. loss: 0.1129
Epoch 40, 100% 	 Train loss: 0.0940 took: 1.99s  Val. loss: 0.1030
Epoch 41, 100% 	 Train loss: 0.0941 took: 1.99s  Val. loss: 0.1035
Epoch 42, 100% 	 Train loss: 0.0921 took: 1.20s  Val. loss: 0.

Epoch 37, 100% 	 Train loss: 0.2578 took: 4.40s  Val. loss: 0.2589
Epoch 38, 100% 	 Train loss: 0.2578 took: 4.44s  Val. loss: 0.2591
Epoch 39, 100% 	 Train loss: 0.2578 took: 4.50s  Val. loss: 0.2584
Epoch 40, 100% 	 Train loss: 0.2578 took: 4.44s  Val. loss: 0.2596
Epoch 41, 100% 	 Train loss: 0.2578 took: 4.53s  Val. loss: 0.2584
Epoch 42, 100% 	 Train loss: 0.2578 took: 4.68s  Val. loss: 0.2582
Epoch 43, 100% 	 Train loss: 0.2578 took: 4.55s  Val. loss: 0.2587
Epoch 44, 100% 	 Train loss: 0.2578 took: 4.62s  Val. loss: 0.2594
Epoch 45, 100% 	 Train loss: 0.2578 took: 4.76s  Val. loss: 0.2578
Epoch 46, 100% 	 Train loss: 0.2578 took: 4.74s  Val. loss: 0.2589
Epoch 47, 100% 	 Train loss: 0.2578 took: 4.72s  Val. loss: 0.2577
Epoch 48, 100% 	 Train loss: 0.2578 took: 4.94s  Val. loss: 0.2585
Epoch 49, 100% 	 Train loss: 0.2578 took: 4.91s  Val. loss: 0.2589
Epoch 50, 100% 	 Train loss: 0.2578 took: 5.03s  Val. loss: 0.2583
Training finished, took 184.38s

Creating dataset...

Training


Training network...
Verbose:  True
Using cuda:  False
Epoch 1, 100% 	 Train loss: 0.2611 took: 1.97s  Val. loss: 0.2580
Epoch 2, 100% 	 Train loss: 0.2605 took: 1.96s  Val. loss: 0.2584
Epoch 3, 100% 	 Train loss: 0.2527 took: 1.93s  Val. loss: 0.2258
Epoch 4, 100% 	 Train loss: 0.2076 took: 1.97s  Val. loss: 0.1777
Epoch 5, 100% 	 Train loss: 0.1798 took: 1.95s  Val. loss: 0.1730
Epoch 6, 100% 	 Train loss: 0.1746 took: 1.93s  Val. loss: 0.1722
Epoch 7, 100% 	 Train loss: 0.1722 took: 1.93s  Val. loss: 0.1665
Epoch 8, 100% 	 Train loss: 0.1698 took: 1.13s  Val. loss: 0.1658
Epoch 9, 100% 	 Train loss: 0.1672 took: 1.14s  Val. loss: 0.1653
Epoch 10, 100% 	 Train loss: 0.1657 took: 1.14s  Val. loss: 0.1607
Epoch 11, 100% 	 Train loss: 0.1645 took: 1.14s  Val. loss: 0.1597
Epoch 12, 100% 	 Train loss: 0.1642 took: 1.13s  Val. loss: 0.1614
Epoch 13, 100% 	 Train loss: 0.1622 took: 1.13s  Val. loss: 0.1675
Epoch 14, 100% 	 Train loss: 0.1610 took: 1.13s  Val. loss: 0.1593
Epoch 15, 100% 	

Epoch 19, 100% 	 Train loss: 0.1880 took: 1.15s  Val. loss: 0.1808
Epoch 20, 100% 	 Train loss: 0.1820 took: 1.16s  Val. loss: 0.1793
Epoch 21, 100% 	 Train loss: 0.1788 took: 1.16s  Val. loss: 0.1731
Epoch 22, 100% 	 Train loss: 0.1753 took: 1.16s  Val. loss: 0.1698
Epoch 23, 100% 	 Train loss: 0.1748 took: 1.15s  Val. loss: 0.1687
Epoch 24, 100% 	 Train loss: 0.1728 took: 1.15s  Val. loss: 0.1701
Epoch 25, 100% 	 Train loss: 0.1718 took: 1.15s  Val. loss: 0.1657
Epoch 26, 100% 	 Train loss: 0.1700 took: 1.15s  Val. loss: 0.1643
Epoch 27, 100% 	 Train loss: 0.1714 took: 1.16s  Val. loss: 0.1653
Epoch 28, 100% 	 Train loss: 0.1686 took: 1.16s  Val. loss: 0.1640
Epoch 29, 100% 	 Train loss: 0.1681 took: 1.17s  Val. loss: 0.1639
Epoch 30, 100% 	 Train loss: 0.1669 took: 1.17s  Val. loss: 0.1653
Epoch 31, 100% 	 Train loss: 0.1674 took: 1.18s  Val. loss: 0.1637
Epoch 32, 100% 	 Train loss: 0.1660 took: 1.18s  Val. loss: 0.1625
Epoch 33, 100% 	 Train loss: 0.1659 took: 1.18s  Val. loss: 0.

In [ ]:
for i in range(10):
    bayes_HP_tuning.step()


Configuration sampled: 
	out_channels :  [6]
	max_pool_size :  2
	n_features :  64
	info_channels :  4
	mask_channels :  8
	hidden_channels :  6
	residual_hidden_dim :  16
	n_residual_layers :  1

Creating dataset...

Training network...
Verbose:  True
Using cuda:  False
Epoch 1, 100% 	 Train loss: 0.2572 took: 1.76s  Val. loss: 0.2580
Epoch 2, 100% 	 Train loss: 0.2564 took: 1.75s  Val. loss: 0.2587
Epoch 3, 100% 	 Train loss: 0.2564 took: 1.74s  Val. loss: 0.2571
Epoch 4, 100% 	 Train loss: 0.2563 took: 1.75s  Val. loss: 0.2576
Epoch 5, 100% 	 Train loss: 0.2563 took: 1.77s  Val. loss: 0.2570
Epoch 6, 100% 	 Train loss: 0.2556 took: 1.75s  Val. loss: 0.2538
Epoch 7, 100% 	 Train loss: 0.2250 took: 1.76s  Val. loss: 0.1867
Epoch 8, 100% 	 Train loss: 0.1696 took: 1.76s  Val. loss: 0.1700
Epoch 9, 100% 	 Train loss: 0.1615 took: 1.43s  Val. loss: 0.1674
Epoch 10, 100% 	 Train loss: 0.1569 took: 1.00s  Val. loss: 0.1619
Epoch 11, 100% 	 Train loss: 0.1517 took: 1.00s  Val. loss: 0.1623

Epoch 15, 100% 	 Train loss: 0.1760 took: 1.76s  Val. loss: 0.1757
Epoch 16, 100% 	 Train loss: 0.1739 took: 1.76s  Val. loss: 0.1735
Epoch 17, 100% 	 Train loss: 0.1715 took: 1.76s  Val. loss: 0.1733
Epoch 18, 100% 	 Train loss: 0.1699 took: 1.74s  Val. loss: 0.1700
Epoch 19, 100% 	 Train loss: 0.1687 took: 1.75s  Val. loss: 0.1697
Epoch 20, 100% 	 Train loss: 0.1684 took: 1.75s  Val. loss: 0.1726
Epoch 21, 100% 	 Train loss: 0.1669 took: 1.77s  Val. loss: 0.1692
Epoch 22, 100% 	 Train loss: 0.1674 took: 1.75s  Val. loss: 0.1671
Epoch 23, 100% 	 Train loss: 0.1651 took: 1.76s  Val. loss: 0.1666
Epoch 24, 100% 	 Train loss: 0.1643 took: 1.77s  Val. loss: 0.1667
Epoch 25, 100% 	 Train loss: 0.1644 took: 1.77s  Val. loss: 0.1693
Epoch 26, 100% 	 Train loss: 0.1651 took: 1.00s  Val. loss: 0.1699
Epoch 27, 100% 	 Train loss: 0.1625 took: 1.00s  Val. loss: 0.1654
Epoch 28, 100% 	 Train loss: 0.1654 took: 1.00s  Val. loss: 0.1675
Epoch 29, 100% 	 Train loss: 0.1620 took: 1.00s  Val. loss: 0.

Epoch 24, 100% 	 Train loss: 0.1606 took: 1.00s  Val. loss: 0.1636
Epoch 25, 100% 	 Train loss: 0.1584 took: 1.00s  Val. loss: 0.1636
Epoch 26, 100% 	 Train loss: 0.1592 took: 1.00s  Val. loss: 0.1650
Epoch 27, 100% 	 Train loss: 0.1571 took: 1.00s  Val. loss: 0.1615
Epoch 28, 100% 	 Train loss: 0.1560 took: 1.00s  Val. loss: 0.1630
Epoch 29, 100% 	 Train loss: 0.1552 took: 1.00s  Val. loss: 0.1614
Epoch 30, 100% 	 Train loss: 0.1545 took: 1.00s  Val. loss: 0.1669
Epoch 31, 100% 	 Train loss: 0.1548 took: 1.00s  Val. loss: 0.1646
Epoch 32, 100% 	 Train loss: 0.1533 took: 1.00s  Val. loss: 0.1600
Epoch 33, 100% 	 Train loss: 0.1516 took: 1.01s  Val. loss: 0.1583
Epoch 34, 100% 	 Train loss: 0.1497 took: 1.10s  Val. loss: 0.1558
Epoch 35, 100% 	 Train loss: 0.1478 took: 1.02s  Val. loss: 0.1571
Epoch 36, 100% 	 Train loss: 0.1455 took: 1.02s  Val. loss: 0.1548
Epoch 37, 100% 	 Train loss: 0.1452 took: 1.03s  Val. loss: 0.1512
Epoch 38, 100% 	 Train loss: 0.1418 took: 1.03s  Val. loss: 0.